---
#### Import Standard Libraries
---

In [1]:
#!pip install pyxlsb

In [2]:


import pandas as pd
import numpy as np
import datetime as dt

import os
import glob
from pathlib import Path
from pyxlsb import open_workbook, convert_date

---
#### Helper Functions
---

In [3]:
def find_latest_file(paths: dict, year: str) -> str:
    """
    function: run_main

    write coms data that has been process using function: process_raw_coms

    :param_tbl: paths, args
    :return:    path string to latest gop file
    """
    rcv_path = paths['rcv_path']
    
    files = get_files(rcv_path,f'**/*GOP*.xlsb')
    if len(files) == 0:
        raise ValueError('No files were returned')
        
    if args.latest:
        max_file_time = datetime.datetime.now() - datetime.timedelta(days=4)
        for f in files:
            print(time.ctime(os.path.getctime(f)))
            if datetime.datetime.strptime(time.ctime(os.path.getctime(f)), "%a %b %d %H:%M:%S %Y") > max_file_time:
                latest_file = f
                print(latest_file)
        csv_file_name = latest_file.split('\\')[-1].split('.')[0] +'.csv'
    return csv_file_name

def get_files(path: str, search_string: str) -> list:
    """
    function: get_files

    write coms data that has been process using function: process_raw_coms

    :param_tbl: path, search_string
    :return:    file_locs as list
    """
    pathlist = Path(path).glob(search_string)
    file_locs = []
    for path in pathlist:
        # because path is object not string
        file_locs.append(str(path))
    return file_locs

---
#### Search Directory and Load Files
---

In [4]:
url = r'E:\_Projects\gop\gop_tabular\gop_input'
pathlist = Path(url).glob('**/GOP*.xlsb')
file_locs = []
for path in pathlist:
     # because path is object not string
     file_locs.append(str(path))
     # print(path_in_str)

files = []
for file in file_locs:
    files.append(file)
files

['E:\\_Projects\\gop\\gop_tabular\\gop_input\\GOP2024.10 Prelim.xlsb']

In [5]:
files = [files[-1]]

In [6]:
files

['E:\\_Projects\\gop\\gop_tabular\\gop_input\\GOP2024.10 Prelim.xlsb']

In [7]:
offer_pf = pd.read_csv(r'E:\_Projects\lookups\class_offerpf_map.csv')
offer_pf.rename(columns = {'class':'Class','group':'Group','offer_pf_wb':'Category','range':'Range',}, inplace = True)
offer_pf.index = offer_pf['Class'].astype('str')

offer_pf = dict(offer_pf.drop(columns='Class'))
print(offer_pf)

{'Group': Class
1611            Servers
1612            Servers
1613        Peripherals
1614            Servers
1615            Servers
               ...     
8101             Others
SR              Modules
SCPM2-5         Modules
SCPM2-30        Modules
Gbxx            Modules
Name: Group, Length: 554, dtype: object, 'category': Class
1611            Servers
1612            Servers
1613        Peripherals
1614            Servers
1615            Servers
               ...     
8101                NaN
SR              Modules
SCPM2-5         Modules
SCPM2-30        Modules
Gbxx            Modules
Name: category, Length: 554, dtype: object, 'Range': Class
1611        NCR Server
1612        NCR Server
1613         NCR Other
1614        NCR Server
1615        NCR Server
               ...    
8101             Other
SR             Modules
SCPM2-5        Modules
SCPM2-30       Modules
Gbxx           Modules
Name: Range, Length: 554, dtype: object, 'prod_grp_wb': Class
1611            Servers

In [8]:
exclusions = pd.read_csv(r'E:\_Projects\gop\gop_tabular\gop_class_exclusions.csv')

In [9]:
mdays = pd.read_csv(r'E:\_Projects\gop\gop_tabular\m_days_acc_period.csv')

In [10]:
rpath = r'E:\_Projects\gop\gop_tabular'
fname = 'Fin Mth.xlsx'
fin_mth = pd.read_excel(f'{rpath}/{fname}', engine="openpyxl")

In [11]:
dropList = [
    'MDAYS',
    'Self Serv 20 - CD',
    'Self Serv 30 - MF',
    'Self Serv 80 - MF',
    'Financial Kiosk',
    ' ATM Total',
    'SelfServe 20',
    'SelfServe 30',
    'SelfServe 80',
    'ATM Daily Rate',
    'SCO Total',
    'SCO Daily Rate',
    'CTO POS',
    'DISPLAYS',
    'SCANNERS',
    'XR POS ',
    'HOSP POS',
    'SERIALIZED KITS',
    'MODULES',
    'HOSP POS MCC',
    'PERIPHERALS',
    'Retail DISPLAYS',
    'Hosp DISPLAYS',
    'OPTIC',
    'NON-Manufacturing Items',
    'OTHER',
    'PRINTERS',
    'Other'
]

sheets = [
    'Ennoconn Bud',
    'Chennai EXP', 
    'Chennai DOM', 
    'Jabil Mex',
    'USI', 
    'CEVA',
    'Shiloh-Bridgestone',
    'Bridgestone',
    'Nashville',
    '2024 EMS MPP'
    'EMS MPP'
]


# -----------------------------------------------------------
# column offsets to ensure alignment of quantities
sheets_col_adj = [
    # 'USI',
    # 'Shiloh-Bridgestone',
    # 'Bridgestone',
    # 'CEVA',
    # 'Chennai EXP', 
    # 'Chennai DOM', 
    # 'Manaus EXP', 
    # 'Manaus DOM'
    ]

# 2020 sites to ignore

site_not_valid_2020 = ['Columbus']


In [12]:
def process_gop_data(f, months):
    '''
    process_gop_data takes a gop file and
    1. processes the xlsb into an array of cell data
    2. extracts the cell values into a data and header array
    3. creates a pandas dataframe of the processed data
    '''
    print(f)
    data = []
    # - need to add sheet information here or can't
    # - differentiate the data
    fname = f.split('\\')[-1]
    with open_workbook(f) as wb:
        # Do stuff with wb
        for name in wb.sheets:
            if name in sheets:
                with wb.get_sheet(name) as sheet:
                    for row in sheet.rows():
                        row.insert(0,fname)
                        row.insert(0, name)
                        data.append(row)
    # process raw data output
    h = []
    d = []
    r = []
    dfs = []
    for headers in data[0]:
        try:
            h.append(headers.v)
        except:
            h.append(headers)    

    for i in range(1,len(data[1:])):
        tmp = []
        for rows in data[i]:
            try:
                tmp.append(rows.v)
            except:
                tmp.append(rows)
        d.append(tmp)

    h = h + [None, None]
    cop = pd.DataFrame(d)
    try:
        cop.iloc[1,58:80] = months
    except:
        print('Failed to Add EMS Data')
    return cop
                

In [13]:
epoch = dt.datetime(1900,1,1)
hdr = ['Site','Scenario', 'Sheetname', 'Product Group', 'Offer PF', 'Range', 'Class', 'SSD', 'Date', 'SSD_Qtr', 'Year', 'MDAYS', 'Qty']

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

mdays = {
    'Jan': {'2025':22,'2024':22,'2023':22, '2019': 10, '2020': 20, '2021': 20, '2022': 20},
    'Feb': {'2025':20,'2024':20,'2023':20, '2019': 20, '2020': 20, '2021': 20, '2022': 20}, 
    'Mar': {'2025':23,'2024':23,'2023':23, '2019': 24, '2020': 25, '2021': 20, '2022': 20}, 
    'Apr': {'2025':23,'2024':23,'2023':23, '2019': 18, '2020': 20, '2021': 20, '2022': 20}, 
    'May': {'2025':21,'2024':21,'2023':21, '2019': 15, '2020': 20, '2021': 20, '2022': 20}, 
    'Jun': {'2025':22,'2024':22,'2023':22, '2019': 24, '2020': 25, '2021': 20, '2022': 20}, 
    'Jul': {'2025':23,'2024':23,'2023':23, '2019': 20, '2020': 20, '2021': 20, '2022': 20}, 
    'Aug': {'2025':21,'2024':21,'2023':21, '2019': 24, '2020': 25, '2021': 20, '2022': 20}, 
    'Sep': {'2025':22,'2024':22,'2023':22, '2019': 20, '2020': 20, '2021': 20, '2022': 20}, 
    'Oct': {'2025':22,'2024':22,'2023':22, '2019': 18, '2020': 20, '2021': 20, '2022': 20}, 
    'Nov': {'2025':21,'2024':21,'2023':21, '2019': 24, '2020': 25, '2021': 20, '2022': 20}, 
    'Dec': {'2025':23,'2024':23,'2023':20, '2019': 20, '2020': 20, '2021': 20, '2022': 20}
}


xlsDates = {
   'Jan': {'2025':45658,'2024':45292, '2023':44927, '2019': 43466, '2020': 43831, '2021': 44197, '2022': 44562},
    'Feb': {'2025':45689, '2024':45323, '2023':44958, '2019': 43497, '2020': 43862, '2021': 44228, '2022': 44593}, 
    'Mar': {'2025':45717, '2024':45352, '2023':44986, '2019': 43525, '2020': 43891, '2021': 44256, '2022': 44621}, 
    'Apr': {'2025':45748, '2024':45383, '2023':45017, '2019': 43556, '2020': 43922, '2021': 44287, '2022': 44652}, 
    'May': {'2025':45778, '2024':45413, '2023':45047, '2019': 43586, '2020': 43952, '2021': 44317, '2022': 44682}, 
    'Jun': {'2025':45809, '2024':45444, '2023':45078, '2019': 43617, '2020': 43983, '2021': 44348, '2022': 44713}, 
    'Jul': {'2025':45839, '2024':45474, '2023':45108, '2019': 43647, '2020': 44013, '2021': 44378, '2022': 44743}, 
    'Aug': {'2025':45870, '2024':45505, '2023':45139, '2019': 43678, '2020': 44044, '2021': 44409, '2022': 44774}, 
    'Sep': {'2025':45901, '2024':45536, '2023':45170, '2019': 43709, '2020': 44075, '2021': 44440, '2022': 44805}, 
    'Oct': {'2025':45931, '2024':45566, '2023':45200, '2019': 43739, '2020': 44105, '2021': 44470, '2022': 44835}, 
    'Nov': {'2025':45962, '2024':45597, '2023':45231, '2019': 43770, '2020': 44136, '2021': 44501, '2022': 44866},
    'Dec': {'2025':45992, '2024':45627, '2023':45261, '2019': 43800, '2020': 44166, '2021': 44531, '2022': 44896} 
}

xlsQtrs = {
    'Jan': {'2025':'2025Q1', '2024':'2024Q1', '2023':'2023Q1', '2019': '2019Q1', '2020': '2020Q1', '2021': '2021Q1', '2022': '2022Q1'},
    'Feb': {'2025':'2025Q1', '2024':'2024Q1', '2023':'2023Q1', '2019': '2019Q1', '2020': '2020Q1', '2021': '2021Q1', '2022': '2022Q1'}, 
    'Mar': {'2025':'2025Q1', '2024':'2024Q1', '2023':'2023Q1', '2019': '2019Q1', '2020': '2020Q1', '2021': '2021Q1', '2022': '2022Q1'}, 
    'Apr': {'2025':'2025Q2', '2024':'2024Q2', '2023':'2023Q2', '2019': '2019Q2', '2020': '2020Q2', '2021': '2021Q2', '2022': '2022Q2'}, 
    'May': {'2025':'2025Q2', '2024':'2024Q2', '2023':'2023Q2', '2019': '2019Q2', '2020': '2020Q2', '2021': '2021Q2', '2022': '2022Q2'}, 
    'Jun': {'2025':'2025Q2', '2024':'2024Q2', '2023':'2023Q2', '2019': '2019Q2', '2020': '2020Q2', '2021': '2021Q2', '2022': '2022Q2'}, 
    'Jul': {'2025':'2025Q3', '2024':'2024Q3', '2023':'2023Q3', '2019': '2019Q3', '2020': '2020Q3', '2021': '2021Q3', '2022': '2022Q3'}, 
    'Aug': {'2025':'2025Q3', '2024':'2024Q3', '2023':'2023Q3', '2019': '2019Q3', '2020': '2020Q3', '2021': '2021Q3', '2022': '2022Q3'}, 
    'Sep': {'2025':'2025Q3', '2024':'2024Q3', '2023':'2023Q3', '2019': '2019Q3', '2020': '2020Q3', '2021': '2021Q3', '2022': '2022Q3'}, 
    'Oct': {'2025':'2025Q4', '2024':'2024Q4', '2023':'2023Q4', '2019': '2019Q4', '2020': '2020Q4', '2021': '2021Q4', '2022': '2022Q4'}, 
    'Nov': {'2025':'2025Q4', '2024':'2024Q4', '2023':'2023Q4', '2019': '2019Q4', '2020': '2020Q4', '2021': '2021Q4', '2022': '2022Q4'}, 
    'Dec': {'2025':'2025Q4', '2024':'2024Q4', '2023':'2023Q4', '2019': '2019Q4', '2020': '2020Q4', '2021': '2021Q4', '2022': '2022Q4'}
}

def factoryOutputv2(d, rng):
    productList = []
    #d.to_excel(r'C:\Users\CP185176\OneDrive - NCR Corporation\project\gop\2019 GOP\10-Oct\whatisthis.xlsx')
    for row in d.values[4:]:
        if row[0] in site_not_valid_2020 and rng[4] in ['2020', '2021']:
            pass
        else:
            try:
                if '2019 EMS MPP' in row[0] and rng[4] == '2019': # EMS does not contain 2020 29th July 2019
                    rng_data = range(rng[2],rng[3])
                    site = row[4]
                    cls_ = str(row[3]).split('.')[0]
                    product_group = offer_pf['Group'].get(cls_)
                    product_category = offer_pf['Category'].get(cls_)   
                    product_range = offer_pf['Range'].get(cls_) 
                    if row[4] is None:
                        ignore = True
                    else:
                        ignore = False
                elif '2018 EMS MPP' in row[0] and rng[4] == '2018': # EMS does contain 2020 v5 onwards
                    rng_data = range(rng[2],rng[3])
                    site = row[4]
                    cls_ = str(row[3]).split('.')[0]
                    product_group = offer_pf['Group'].get(cls_)
                    product_category = offer_pf['Category'].get(cls_)   
                    product_range = offer_pf['Range'].get(cls_) 
                    if row[4] is None:
                        ignore = True
                    else:
                        ignore = False
                elif '2021 EMS MPP' in row[0] and rng[4] == '2021': # EMS does contain 2021 v5 onwards
                    rng_data = range(rng[2],rng[3])
                    site = row[4]
                    cls_ = str(row[3]).split('.')[0]
                    product_group = offer_pf['Group'].get(cls_)
                    product_category = offer_pf['Category'].get(cls_)   
                    product_range = offer_pf['Range'].get(cls_) 
                    if row[4] is None:
                        ignore = True
                    else:
                        ignore = False
                elif '2022 EMS MPP' in row[0] and rng[4] == '2022': # EMS does contain 2021 v5 onwards
                    rng_data = range(rng[2],rng[3])
                    site = row[4]
                    cls_ = str(row[3]).split('.')[0]
                    product_group = offer_pf['Group'].get(cls_)
                    product_category = offer_pf['Category'].get(cls_)   
                    product_range = offer_pf['Range'].get(cls_) 
                    if row[4] is None:
                        ignore = True
                    else:
                        ignore = False
                elif '2023 EMS MPP' in row[0] and rng[4] == '2023': # EMS does contain 2021 v5 onwards
                    rng_data = range(rng[2],rng[3])
                    site = row[4]
                    cls_ = str(row[3]).split('.')[0]
                    product_group = offer_pf['Group'].get(cls_)
                    product_category = offer_pf['Category'].get(cls_)   
                    product_range = offer_pf['Range'].get(cls_) 
                    if row[4] is None:
                        ignore = True
                    else:
                        ignore = False
                        
                elif '2024 EMS MPP' in row[0] and rng[4] == '2024': # EMS does contain 2021 v5 onwards
                    rng_data = range(rng[2],rng[3])
                    site = row[4]
                    cls_ = str(row[3]).split('.')[0]
                    product_group = offer_pf['Group'].get(cls_)
                    product_category = offer_pf['Category'].get(cls_)   
                    product_range = offer_pf['Range'].get(cls_) 
                    if row[4] is None:
                        ignore = True
                    else:
                        ignore = False
                 
                else:
                    rng_data = range(rng[0],rng[1])
                    site = row[0]
                    cls_ = str(row[2]).split('.')[0]
                    product_group = offer_pf['Group'].get(cls_)
                    product_category = offer_pf['Category'].get(cls_)   
                    product_range = offer_pf['Range'].get(cls_)   
                    ignore = False
                    
                    
                    

                if not ignore:
                    for i in rng_data:
                        if site in sheets_col_adj and rng[4] == '2020':
                            month = str(d.values[1][i+1]).strip()
                        else:
                            month = str(d.values[1][i]).strip()
                        if month in months:
                            mday = mdays[month][rng[-1]]
                            dt_value = epoch + dt.timedelta(xlsDates[month][rng[-1]] - 2) #need to take 2days off to align excel date with calendar date
                            ssq_qtr = xlsQtrs[month][rng[-1]]
                            productList.append([site, row[1], row[0], product_group, 
                                                product_category, product_range, cls_, month, 
                                                dt_value, ssq_qtr, rng[-1], mday, row[i]])
            except:
                pass
    return productList


gop_raw = []


yr18Start = 2
yr18End = 16

yr19Start = 15
yr19End = 29

# yr20Start = 2
# yr20End = 15 ## edit this number dependent on TLS

yr21Start = 15
yr21End = 29 ## edit this number dependent on TLS

yr22Start = 2 #29
# # print(yr22start)
yr22End = 16 #43 ## edit this number dependent on TLS

yr23Start = 16 #43
yr23End = 30 #57 ## edit this number dependent on TLS

yr24Start = 30 #43
yr24End = 44

yr25Start = 44 #43
yr25End = 58

for f in files[-1:]:
    gop = process_gop_data(f, months)
    # rng2019 = [yr19Start, yr19End, 58, 70, '2019']   # [m1, m12, ems_m1, ems_m12, year]
#     rng2020 = [yr20Start, yr20End, 58, 70, '2020']   # [m1, m12, ems_m1, ems_m12, year]
#     rng2021 = [yr21Start, yr21End, 58, 70, '2021']   # [m1, m12, ems_m1, ems_m12, year]
    rng2022 = [yr22Start, yr22End, 58, 80, '2022']   # [m1, m12, ems_m1, ems_m12, year]
#     print(rng2022)
    rng2023 = [yr23Start, yr23End, 58, 80, '2023']   # [m1, m12, ems_m1, ems_m12, year]
    print(rng2023)
    rng2024 = [yr24Start, yr24End, 58, 80, '2024']   # [m1, m12, ems_m1, ems_m12, year]
    rng2025 = [yr25Start, yr25End, 58, 80, '2025']   # [m1, m12, ems_m1, ems_m12, year]
    #gop_raw.append(factoryOutputv2(gop,rng2018))
    # gop_raw.append(factoryOutputv2(gop,rng2019))
#     gop_raw.append(factoryOutputv2(gop,rng2020))
#     gop_raw.append(factoryOutputv2(gop,rng2021))
    gop_raw.append(factoryOutputv2(gop,rng2022))
    print(gop_raw)
    gop_raw.append(factoryOutputv2(gop,rng2023))
    gop_raw.append(factoryOutputv2(gop,rng2024))
    gop_raw.append(factoryOutputv2(gop,rng2025))
# completion signal
print('Process complete')

E:\_Projects\gop\gop_tabular\gop_input\GOP2024.10 Prelim.xlsb
Failed to Add EMS Data
[16, 30, 58, 80, '2023']
[[['Ennoconn Bud', 'GOP2024.10 Prelim.xlsb', 'Ennoconn Bud', 'ATM', 'CD', 'NCR Selfserv CD', '6623', 'Jan', datetime.datetime(2022, 1, 1, 0, 0), '2022Q1', '2022', 20, 137.0], ['Ennoconn Bud', 'GOP2024.10 Prelim.xlsb', 'Ennoconn Bud', 'ATM', 'CD', 'NCR Selfserv CD', '6623', 'Feb', datetime.datetime(2022, 2, 1, 0, 0), '2022Q1', '2022', 20, 380.0], ['Ennoconn Bud', 'GOP2024.10 Prelim.xlsb', 'Ennoconn Bud', 'ATM', 'CD', 'NCR Selfserv CD', '6623', 'Mar', datetime.datetime(2022, 3, 1, 0, 0), '2022Q1', '2022', 20, 422.0], ['Ennoconn Bud', 'GOP2024.10 Prelim.xlsb', 'Ennoconn Bud', 'ATM', 'CD', 'NCR Selfserv CD', '6623', 'Apr', datetime.datetime(2022, 4, 1, 0, 0), '2022Q2', '2022', 20, 283.0], ['Ennoconn Bud', 'GOP2024.10 Prelim.xlsb', 'Ennoconn Bud', 'ATM', 'CD', 'NCR Selfserv CD', '6623', 'May', datetime.datetime(2022, 5, 1, 0, 0), '2022Q2', '2022', 20, 322.0], ['Ennoconn Bud', 'GOP20

In [14]:
gop.to_excel(r'E:\_Projects\gop\gop_tabular\whatisthis.xlsx')

In [15]:
gop_out = []

for tls in gop_raw:
    t = pd.DataFrame(tls, columns=hdr)
    t = t[(~t.Qty.isna()) & (t.SSD != 'Total') & (~t.Class.str.contains('rate')) & (~t.Class.str.contains('Total')) & (~t.Class.str.contains('Range')) & (~t.Class.str.contains('Class'))]
    gop_out.append(t)
    
for i in range(len(gop_out)):
    gop_out[i] = gop_out[i][~gop_out[i].Class.isin(dropList)]
gop_out[i].to_excel(r'E:\_Projects\gop\gop_tabular\Check1.xlsx')
print(gop_out)

[              Site                Scenario     Sheetname Product Group  \
0     Ennoconn Bud  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM   
1     Ennoconn Bud  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM   
2     Ennoconn Bud  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM   
3     Ennoconn Bud  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM   
4     Ennoconn Bud  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM   
...            ...                     ...           ...           ...   
4051     Jabil Mex  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules   
4052     Jabil Mex  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules   
4053     Jabil Mex  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules   
4054     Jabil Mex  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules   
4055     Jabil Mex  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules   

     Offer PF            Range Class  SSD       Date SSD_Qtr  Year  MDAYS  \
0          CD  NCR Selfserv CD  6

In [16]:
gop_all = pd.DataFrame()
gop_all = gop_all.append(gop_out)

gop_all = gop_all.merge(fin_mth, how='left', on='Date')

gop_all.to_excel(r'E:\_Projects\gop\gop_tabular\lev1.xlsx')

gop_all['Site'] = gop_all['Site'].apply(lambda x: str(x)[:3].upper())

current_year = dt.datetime.today().year - 1

gop_all['Accounting Period'] = gop_all['Accounting Period'].astype(str)

gop_all.to_excel(r'E:\_Projects\gop\gop_tabular\lev2.xlsx')
print(gop_all)
# gop_all = gop_all[(gop_all['Year'].astype(int) >= current_year) & (~gop_all['Offer PF'].isna())]
# print(gop_all)

C:\Users\ja385000\AppData\Local\Temp\2\ipykernel_24596\2663922637.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gop_all = gop_all.append(gop_out)


     Site                Scenario     Sheetname Product Group Offer PF  \
0     ENN  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM       CD   
1     ENN  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM       CD   
2     ENN  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM       CD   
3     ENN  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM       CD   
4     ENN  GOP2024.10 Prelim.xlsb  Ennoconn Bud           ATM       CD   
...   ...                     ...           ...           ...      ...   
3872  JAB  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules  Modules   
3873  JAB  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules  Modules   
3874  JAB  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules  Modules   
3875  JAB  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules  Modules   
3876  JAB  GOP2024.10 Prelim.xlsb     Jabil Mex       Modules  Modules   

                Range Class  SSD       Date SSD_Qtr  Year  MDAYS    Qty  \
0     NCR Selfserv CD  6623  Jan 202

In [17]:


#fname = files[0].split('\\')[-1].split('.')[0].replace(' ','_').lower() + '_tabular'
fname = files[0].split('\\')[-1].replace('.xlsb','_').lower() + 'tabular'
#fname = fname.astype(str).str.replace('.xlsb_','')
#fname = files[0]+'_tabular'
fname

'gop2024.10 prelim_tabular'

In [18]:
gop_snd = gop_all.merge(exclusions[['Sheetname', 'Class', 'Exclude']], how='left', on=['Sheetname', 'Class'], validate='m:1')

In [19]:
gop_snd['Exclude'].fillna('No', inplace=True)
gop_snd.rename(columns={'Class': 'GOP Class'}, inplace=True)
gop_snd['Class'] = gop_snd['GOP Class'].apply(lambda x: str(x)[:4])
# gop_snd.to_excel(r'E:\_Projects\gop\gop_tabular\lev3.xlsx')

In [20]:
#gop_snd['Site'] = gop_snd['Site'].replace('CEV', 'USL').replace('USI', 'USL')
gop_snd['Site'] = gop_snd['Site'].replace('CEV', 'USL').replace('BRI', 'USL').replace('NAS', 'USL').replace('ENN', 'BUD')

In [21]:
out_hdr = ['Site','Scenario', 'Sheetname', 'Product Group', 'Offer PF', 
           'Range', 'Class', 'GOP Class', 'SSD', 'Date', 'Accounting Period', 
           'SSD_Qtr', 'Year', 'MDAYS', 'Qty', 'Exclude', 'ExcludeWSP']

In [22]:
fname

'gop2024.10 prelim_tabular'

In [23]:
try:
    gop_snd.drop(columns='Exclude', inplace=True)
except KeyError:
    print("Column 'Exclude' not in dataframe")
    
gop_snd['GOP Class'] = gop_snd['GOP Class'].astype(str)
gop_snd['Accounting Period'] = gop_snd['Accounting Period'].astype(int)
excl_list = pd.read_excel(r'E:\_Projects\gop\gop_tabular\gop_exclusion_list_v2.xlsx', engine='openpyxl')
gop_snd = gop_snd.merge(excl_list,how='left',on=['Sheetname', 'GOP Class', 'Accounting Period'])

In [24]:
map_path = r'E:\_Projects\lookups\class_offerpf_map.csv'
wsp = pd.read_csv(map_path)
wsp.rename(columns = {'class':'Class','offer_pf_wb':'Offer Prod Portfolio','prod_grp_wb':'Grp','range':'Prod Range',}, inplace = True)
wsp = wsp[['Class', 'Prod Range', 'Offer Prod Portfolio','Grp', 'Flag_Discon','Flag_Exclude', 'Flag_USI ', 'Show/Hide']]
wsp.columns


Index(['Class', 'Prod Range', 'Offer Prod Portfolio', 'Grp', 'Flag_Discon',
       'Flag_Exclude', 'Flag_USI ', 'Show/Hide'],
      dtype='object')

In [25]:
wsp['Class'] = wsp['Class'].astype(str)

In [26]:
gop_snd_final = gop_snd[out_hdr].merge(wsp, how='left', on='Class', validate='m:1')

In [27]:
gop_snd_final['Exclude'] = gop_snd_final['Exclude'].fillna('No')
gop_snd_final['ExcludeWSP'] = gop_snd_final['ExcludeWSP'].fillna('No')

In [28]:
fname

'gop2024.10 prelim_tabular'

In [29]:
mdays = pd.read_csv(r'E:\_Projects\gop\gop_tabular\m_days_acc_period.csv')

In [30]:
gop_snd_final['Accounting Period'] = gop_snd_final['Accounting Period'].astype(str)
mdays['Accounting Period'] = mdays['Accounting Period'].astype(str)

In [31]:
gop_snd_final.drop(columns=['MDAYS'], inplace=True, errors='ignore')
gop_snd_final = gop_snd_final.merge(mdays, how='left', on=['Site', 'Accounting Period'], validate='m:1')
gop_snd_final['Date'] = gop_snd_final['Date'].dt.strftime('%m/%d/%Y')
print (gop_snd_final['Date'])
gop_snd_final.to_csv("data.csv")

0       01/01/2022
1       02/01/2022
2       03/01/2022
3       04/01/2022
4       05/01/2022
           ...    
3872    08/01/2025
3873    09/01/2025
3874    10/01/2025
3875    11/01/2025
3876    12/01/2025
Name: Date, Length: 3877, dtype: object


In [32]:
# gop_snd_final['Class'] = np.where(gop_snd_final['GOP Class']=='7360','7360S',gop_snd_final['Class'])
# gop_snd_final['Class'] = np.where(gop_snd_final['GOP Class']=='7360 R6N','7360N',gop_snd_final['Class'])
# gop_snd_final['Class'] = np.where(gop_snd_final['GOP Class']=='7360 WM','7360J',gop_snd_final['Class'])

In [33]:
gop_snd_final.to_excel(f'E:\\_Projects\\gop\\gop_tabular\\gop_out//{fname}.xlsx', index = False)
# gop_snd_final.to_excel(f'E:\\_Projects\\gop\\gop_tabular\\gop_out\\To Bi//{fname}.xlsx', index = False)
# gop_snd_final.to_excel(f'E:\\_Projects\\gop\\gop_tabular\\gop_out\\gop_tabluar.xlsx', index = False)
# gop_snd_final.to_excel(f'\\Dayorg1\\ORGSHARE\\TEAMS\\ERP Shared Folder\\Global S&OP\\Daily Order Cover - Global\\gop\\{fname}.xlsx', index = False)
# gop_snd_final.to_csv(f'\\Dayorg1\\ORGSHARE\\TEAMS\\ERP Shared Folder\\Global S&OP\\Daily Order Cover - Global\\gop\\gop_tabular.csv', index = False)

In [34]:
def leap_year(yr):
    if yr % 4 != 0:
        print('Not leap year')
    elif yr % 100 != 0:
        print('Leap year')
    elif yr % 400 != 0:
        print('Not leap year')
    else:
        print('Leap year')

In [35]:
dict(gop_snd['Sheetname'].drop_duplicates())

{0: 'Ennoconn Bud', 255: 'Chennai EXP', 691: 'Chennai DOM', 755: 'Jabil Mex'}

In [36]:
sitename_dict = {'Ennoconn Bud': 'Ennoconn Bud',
 'Chennai EXP': 'Chennai Total',
 'Chennai DOM': 'Chennai Total',
 'Jabil Mex': 'Jabil Mex',
 'Manaus EXP': 'Manaus Total',
 'Manaus DOM': 'Manaus Total',
 'USI': 'USL',
 'CEVA': 'USL',
 'Bridgestone': 'USL',
 '2019 EMS MPP': '2019 EMS MPP',
 'Columbus': 'Columbus',
 'Midland': 'Midland',
 '2020 EMS MPP': '2020 EMS MPP'}

In [37]:
gop_ord = gop_snd.copy()

In [38]:
gop_ord['Sheetname'].drop_duplicates()

0      Ennoconn Bud
255     Chennai EXP
691     Chennai DOM
755       Jabil Mex
Name: Sheetname, dtype: object

In [39]:
gop_ord.columns

Index(['Site', 'Scenario', 'Sheetname', 'Product Group', 'Offer PF', 'Range',
       'GOP Class', 'SSD', 'Date', 'SSD_Qtr', 'Year', 'MDAYS', 'Qty',
       'Aged Flag', 'Month Flag', 'Accounting Period', 'Week', 'WeekNum',
       'Start Date', 'End Date', 'Class', 'Exclude', 'ExcludeWSP'],
      dtype='object')

In [40]:
gop_ord['Sheetname'] = gop_ord['Sheetname'].apply(lambda x: sitename_dict.get(x))

In [41]:
def replace_group(df):
    new_rows = []
    for row in df.values:
        if row[7] == '7702 SCO Head' or row[7] == '7703 SCO Head':
            row[3] = 'POS'
            row[4] = 'XR POS'
            row[5] = 'NCR POS'
            row[-2] = 'No'
        new_rows.append(row)
    df = pd.DataFrame(new_rows, columns=df.columns)
    return df
new_df = replace_group(gop_ord)

In [42]:


#grp_cols = ['Site','Scenario', 'Sheetname', 'Product Group', 'Offer PF', 
           #'Range', 'Class', 'SSD', 'Date', 'Accounting Period', 
           #'SSD_Qtr', 'Year', 'MDAYS', 'Exclude']
#gop_ord = gop_ord[out_hdr].groupby(grp_cols).sum(by={'Qty': sum}).reset_index()

In [43]:
out_hdr = ['Site','Scenario', 'Sheetname', 'Offer PF', 'Class', 'SSD', 'Date', 'Accounting Period', 
            'Year', 'MDAYS', 'Qty', 'Exclude']

In [44]:
## Exclude == 'Yes' to POS, ATM and SCO
## USI and CEVA == USL
## Modules for 7702 and 7703 need to be removed
## double-check 7603 on USL --- should be in the data
#gop_ord[out_hdr][(~gop_ord['Offer PF'].isin(['Modules'])) & (~gop_ord['Class'].isin([7702, 7703]))].to_excel(f'C:\\Users\\CP185176\\OneDrive - NCR Corporation\\project\\gop\\for_sit_tool.xlsx', index = False)